In [4]:
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import *

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

base_model=MobileNetV2(weights='imagenet',include_top=False) #imports the MobileNetV2 model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='softmax')(x) #final layer with softmax activation for N classes

model=Model(inputs=base_model.input,outputs=preds) #specify the inputs and outputs

num_trainable_layers = 20
for layer in model.layers[:num_trainable_layers]:
    layer.trainable=False
for layer in model.layers[num_trainable_layers:]:
    layer.trainable=True

#Training set
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator=train_datagen.flow_from_directory('C:/Users/Schiller/PyImageSearch/R-CNN_object_detection/rcnn_object_detection/dataset/',
                                                target_size=(224,224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical',
                                                shuffle=True)

#Training set
val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator=val_datagen.flow_from_directory('C:/Users/Schiller/PyImageSearch/R-CNN_object_detection/rcnn_object_detection/dataset/',
                                                target_size=(224,224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical',
                                                shuffle=True)


model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

step_size_train=train_generator.n//train_generator.batch_size
step_size_val=val_generator.n//val_generator.batch_size
model.fit_generator(generator=train_generator,
                  steps_per_epoch=step_size_train,
                  epochs=100, validation_data=val_generator,
                  validation_steps=step_size_val)

FileNotFoundError: ignored